[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1b-sg2SfnlSLUReyNDVnqtfpYFI_jf3i0?usp=sharing)
# Техническая часть

In [ ]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [ ]:
!pip install -q sentence-transformers
!pip install -q razdel

     |████████████████████████████████| 71kB 1.7MB/s 
     |████████████████████████████████| 1.1MB 7.5MB/s 
     |████████████████████████████████| 3.0MB 27.7MB/s 
     |████████████████████████████████| 890kB 34.6MB/s 
     |████████████████████████████████| 1.1MB 36.3MB/s 


In [ ]:
!git clone https://github.com/DanilDmitriev1999/extractive_summarization

Cloning into 'extractive_summarization'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 32 (delta 9), reused 28 (delta 8), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [ ]:
from itertools import combinations
import networkx as nx
import json
import random
from tqdm import tqdm

from sentence_transformers import SentenceTransformer

import numpy as np
from scipy.linalg import norm

from nltk.translate.bleu_score import corpus_bleu
import razdel

from pprint import pprint

Весь код лежит в [моём](https://github.com/DanilDmitriev1999/extractive_summarization) репозитории

In [ ]:
from extractive_summarization.utils import *
from extractive_summarization.text_rank import *

# Данные

In [ ]:
train_records = read_gazeta_records("gazeta_train.txt", 52400)
val_records = read_gazeta_records("gazeta_val.txt", 5265)
test_records = read_gazeta_records("gazeta_test.txt", 5770)

100%|██████████| 5770/5770 [00:00<00:00, 13982.96it/s]


In [ ]:
pprint(train_records[219])

{'date': '2018-08-22 08:01:56',
 'summary': 'Перед контрольными прокатами, которые пройдут 9 сентября в '
            'Москве, российские фигуристы примут участие в сборе в Новогорске. '
            'Любители этого вида спорта ждут не только возвращения спортсменов '
            'на лед, но и встречи серебряного призера Олимпийских игр 2018 '
            'года Евгении Медведевой с бывшим тренером Этери Тутберидзе.',
 'text': 'До старта нового сезона в фигурном катании остается все меньше '
         'времени. Первый этап Гран-при пройдет в американском городе Эверетт '
         'с 19 по 21 октября. Перед этим представители сборной России примут '
         'участие в контрольных прокатах, которые пройдут 9 сентября в Москве '
         'на арене «Мегаспорт». Еще раньше спортсмены примут участие в сборе в '
         'Новогорске. Генеральный директор Федерации фигурного катания на '
         'коньках России (ФФККР) Александр Коган заявил, что в Новогорск '
         'прибудет двукратный сере

# mDistilBert + TextRank

In [ ]:
model_name = 'distilbert-multilingual-nli-stsb-quora-ranking'
model = TextRank(model_name)

100%|██████████| 501M/501M [00:21<00:00, 22.9MB/s]


In [ ]:
model.get_summary(train_records[14])


Count: 1
Полный текст:
('В пятницу, 27 мая, в столичном регионе давление повысится до 749 мм рт. ст. Как пояснили «Газете.Ru» в центре «Фобос» , из-за этого солнце сильнее '
 'прогреет воздух: днем ожидается +22…+27°С (ночью +9…+14°С). Атмосфера станет неустойчивой, а это, в свою очередь, вызовет короткие дожди и грозы — '
 'преимущественно днем. Из-за духоты, которая всегда возникает перед грозой, жара будет переноситься еще хуже, так что всем страдающим от '
 'сердечно-сосудистых заболеваний или болезней органов дыхания стоит быть втройне осторожнее. Ветер ночью переменных направлений, слабый, днем '
 'восточный, северо-восточный, 3–8 м/с, при грозе — порывистый. Последний уикенд весны поддержит погодный тренд: синоптическая ситуация в Москве '
 'существенно не изменится. Оба дня переменная облачность, днем местами кратковременный дождь, не исключена гроза. Давление высокое — 751 мм рт. '
 'ст., ветер в субботу южный, в воскресенье — северо-восточный, оба дня скорость одинаковая, 3–

In [ ]:
model.get_summary(train_records[4321])


Count: 1
Полный текст:
('Ситуацию в Белоруссии рассмотрят на заседании Совета ООН по правам человека. С таким предложением выступил Евросоюз. «Одним из главных приоритетов '
 'ЕС в перспективе сессии Совета ООН по правам человека, которая состоится в июне, будет ситуация с правами человека в Белоруссии, которая '
 'продолжает вызывать горячую обеспокоенность», – сказано в специальном постановлении Совета ЕС , принятом в связи с подготовкой регулярной сессии '
 'Совета ООН. Документ был принят в понедельник в Брюсселе министрами иностранных дел стран ЕС. В нем также говорится, что Евросоюз считает '
 'приоритетным на ближайшей сессии обсудить и ситуацию с правами человека в Сирии, где нарушается международное право. Также Совет ЕС обращает '
 'внимание Совета ООН на критическое положение с правами человека в КНДР. Евросоюз совместно с Японией предложит на сессии создать независимый '
 'механизм расследования ситуации с обеспечением прав человека в Северной Корее. Наконец, ЕС намерен ак

In [ ]:
model.get_summary(train_records[2900])

100%|██████████| 501M/501M [00:21<00:00, 23.3MB/s]



Count: 1
Полный текст:
('«Подтверждаю, что губернатор прилетел», — подтвердил ТАСС информацию о возвращении губернатора неназванный источник, близкий к Тулеев у. '
 'Губернатор был доставлен в Кемеровскую область чартерным рейсом. Его встречал автомобиль скорой помощи и замгубернатора по медицине Валерий Цой. '
 'Самочувствие Амана Тулеева называют хорошим. «Состояние губернатора стабильное, он чувствует себя значительно лучше. Сейчас он будет проходить '
 'реабилитационный курс в регионе, будет включаться в работу, в том числе планирует проводить рабочие встречи с подчиненными», — добавил собеседник '
 'агентства. Официально Тулеев находился в отпуске начиная с 22 мая. Изначально глава Кемеровской области взял отпуск на 10 дней, но после срок '
 'неоднократно продлевался. Все это время обязанности главы региона исполнял первый заместитель губернатора Владимир Чернов. Операция на '
 'позвоночнике губернатора длилась 9 часов и была проведена в клинике, расположенной в Германии. Врачи п

Есть поддержка своих текстов.

In [ ]:
model.get_summary(train_records[2900]['text'])

Полный текст:
('«Подтверждаю, что губернатор прилетел», — подтвердил ТАСС информацию о возвращении губернатора неназванный источник, близкий к Тулеев у. '
 'Губернатор был доставлен в Кемеровскую область чартерным рейсом. Его встречал автомобиль скорой помощи и замгубернатора по медицине Валерий Цой. '
 'Самочувствие Амана Тулеева называют хорошим. «Состояние губернатора стабильное, он чувствует себя значительно лучше. Сейчас он будет проходить '
 'реабилитационный курс в регионе, будет включаться в работу, в том числе планирует проводить рабочие встречи с подчиненными», — добавил собеседник '
 'агентства. Официально Тулеев находился в отпуске начиная с 22 мая. Изначально глава Кемеровской области взял отпуск на 10 дней, но после срок '
 'неоднократно продлевался. Все это время обязанности главы региона исполнял первый заместитель губернатора Владимир Чернов. Операция на '
 'позвоночнике губернатора длилась 9 часов и была проведена в клинике, расположенной в Германии. Врачи приняли реш